In [12]:
def compute_glcm_descriptor(image_path):
    # Read and preprocess image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        return None
    
    # Configuration 
    distance = [3]  
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # 0°, 45°, 90°, 135°
    properties = ['contrast', 'correlation', 'energy', 'homogeneity']  
    
    # Compute GLCM
    glcm = graycomatrix(image, 
                       distances=distance,
                       angles=angles,
                       symmetric=True,
                       normed=True)
    
    # Extract features for each direction
    features = []
    for prop in properties:
        feature = graycoprops(glcm, prop).flatten()
        features.extend(feature)
    
    return np.array(features)

In [15]:
def process_directory(base_path):
    all_features = []
    all_labels = []
    
    image_paths = []
    for root, dirs, files in os.walk(base_path):
        if 'Negative' in root:
            continue
        category = os.path.basename(root)
        for file in files:
            if file.endswith(('.png')):
                image_paths.append((os.path.join(root, file), category))
    
    for image_path, category in tqdm(image_paths, desc="Processing GLCM"):
        features = compute_glcm_descriptor(image_path)
        if features is not None:
            all_features.append(features)
            all_labels.append(category)
    
    df = pd.DataFrame(all_features)
    df['category'] = all_labels
    df['label'] = df['category'].astype('category').cat.codes
    return df




In [17]:
base_path = '/home/duyle/Downloads/Rice_photos'
df = process_directory(base_path)

Processing GLCM:   0%|          | 0/10088 [00:00<?, ?it/s]

Processing GLCM: 100%|██████████| 10088/10088 [04:07<00:00, 40.72it/s]


In [19]:
df.to_csv('rice_glcm_features.csv', index=False)